In [1]:
import pandas as pd;
from pandas import DataFrame, Series;
import datetime;
import json;

In [2]:
import sys
sys.path.insert(0, '../DAL')

import dataService
service = dataService.DataService()
r_r = service.review_dates()

Drill is active: True
started on: 2016-04-03 13:50:40.366210
finished on: 2016-04-03 13:55:35.155369


In [3]:
#Compute date; remove unnecessary columns & reviews
# r_r = DataFrame(dataService.DataService.review_dates()[:100])
lastDay = datetime.datetime(2015, 12, 24)
r_r['reference_date'] = r_r['reference_date'].map(lambda x: (lastDay - datetime.datetime.strptime(x, '%Y-%m-%d')).days)
r_r['review_date'] = r_r['review_date'].map(lambda x: (lastDay - datetime.datetime.strptime(x, '%Y-%m-%d')).days)

In [4]:
r_r.head()

,reference_id,reference_date,authority_stars,authority_id,review_id,review_date,user_id,validation_token
0,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,fpJ_Bgho7G3E-_WWaMJpjA,2029,YKDw2WGj-MQjz7eRsb5FHg,9OZH1Ecw-qUkCW5MS0NefAYKDw2WGj-MQjz7eRsb5FHg
1,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,nuZlz-65cXIsLZPGk6EwDw,1807,_CtkjL-e6O_lMShkN6HuSA,9OZH1Ecw-qUkCW5MS0NefA_CtkjL-e6O_lMShkN6HuSA
2,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,DtWoMT9VcFX9ycdZFpZoow,1510,0kRiaCs6damnnCpjPNOqRw,9OZH1Ecw-qUkCW5MS0NefA0kRiaCs6damnnCpjPNOqRw
3,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,M28dSKnyInjQu0I9NJlfig,1000,RJgfj-6Uq-lyiPJJcyCVlQ,9OZH1Ecw-qUkCW5MS0NefARJgfj-6Uq-lyiPJJcyCVlQ
4,d3w57QihQsYNDmZi3x1x8w,507,3,9OZH1Ecw-qUkCW5MS0NefA,QRRumDA7QEUGwQ2sD_KswA,927,fTg6Rz2Rs1LLYN_U7TJbcQ,9OZH1Ecw-qUkCW5MS0NefAfTg6Rz2Rs1LLYN_U7TJbcQ


In [5]:
r_r['days_since_review'] = r_r.apply(lambda x: x['reference_date'] - x['review_date'], axis=1)
r_r['time_distribution'] = r_r.apply(
    lambda x: 1 if (x['days_since_review'] > 0) 
    else (
        0 if (x['days_since_review'] + x['reference_date'] >= 0)
        else -1
    ), axis=1)
# then -1)
r_r = r_r[['authority_id', 'reference_id', 'user_id', 'review_id', 'authority_stars', 'time_distribution', 'validation_token']]
r_r = DataFrame(r_r.loc[r_r['time_distribution'] >= 0])

In [6]:
#Add distances
u_u = pd.read_json('../full_dataset/user_distance_matrix.json')
u_u.head()

,distance,friend_id,user_id
0,1,-4SyV4ixKhtexbStYy6-RQ,eQ4PN9G8Q9ipqBxFrAS7YQ
1,1,FUd5OsFEURn2Qzg10XGeAQ,eQ4PN9G8Q9ipqBxFrAS7YQ
10,1,azH9azGyJxL54wclFqr-rg,eQ4PN9G8Q9ipqBxFrAS7YQ
100,1,h4RkUrmcsMc0VTUCdaY4_g,0mVScF9nfPVIkS2OlbvpNw
1000,1,44-fkNhm_1WkkWbD71nHFQ,MOdILSTOFnq0iXHN9l8_zA


In [ ]:
u_u['validation_token'] = u_u.apply(lambda x: x['user_id'] + x['friend_id'], axis = 1)

In [ ]:
r_r = pd.merge(r_r, u_u, on='validation_token')
u_u = None

In [ ]:
r_r.head()

In [ ]:
distribution = r_r.groupby(['authority_id', 'time_distribution', 'distance', 'authority_stars']).size().reset_index()

In [ ]:
len (distribution)

In [ ]:
distribution.head()

In [ ]:
distribution.to_json('../full_dataset/visit_distribution_authorityid.json')

In [ ]:
distribution2 = r_r.groupby(['reference_id', 'time_distribution', 'distance', 'authority_stars']).size().reset_index()
distribution2.to_json('../full_dataset/visit_distribution_referenceid.json')

In [ ]:
len (distribution2)

In [ ]:
distribution2.head()